In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def preprocess_image(image_path, img_width, img_height):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    img = img.astype('float32') / 255.0
    return img

def load_dataset(data_dir, img_width, img_height):
    X = []
    y = []
    emotion_classes = sorted(os.listdir(data_dir))
    for i, emotion in enumerate(emotion_classes):
        emotion_dir = os.path.join(data_dir, emotion)
        if os.path.isdir(emotion_dir):  # Check if the item is a directory
            for image_name in os.listdir(emotion_dir):
                image_path = os.path.join(emotion_dir, image_name)
                try:
                    img = cv2.imread(image_path)
                    if img is not None:  # Check if the image was loaded successfully
                        img = cv2.resize(img, (img_width, img_height))
                        img = img.astype('float32') / 255.0
                        X.append(img)
                        y.append(i)
                    else:
                        print(f"Warning: Skipping invalid image file: {image_path}")
                except Exception as e:
                    print(f"Error: {e}, Skipping image: {image_path}")

    X = np.array(X)
    y = np.array(y)
    y = to_categorical(y, num_classes=len(emotion_classes))
    return X, y

def inception_block(x, filters):
    # 1x1 Convolution
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    # 3x3 Convolution
    conv3x3 = Conv2D(filters[1], (3, 3), padding='same', activation='relu')(x)

    # 5x5 Convolution
    conv5x5 = Conv2D(filters[2], (5, 5), padding='same', activation='relu')(x)

    # 3x3 MaxPooling followed by 1x1 Convolution
    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    conv1x1_pool = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(pool)

    # Concatenate all the outputs
    inception_output = concatenate([conv1x1, conv3x3, conv5x5, conv1x1_pool], axis=-1)
    return inception_output


def build_inception_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # First Convolutional layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # First Inception block
    x = inception_block(x, [64, 128, 32, 32])
    
    # Add more Inception blocks here if needed

    # Fully Connected layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


# Load and preprocess the entire dataset
data_dir = '/Users/eunjincho/Documents/workspaces/data_geeks/datasets/'
img_width, img_height = 224, 224

X, y = load_dataset(data_dir, img_width, img_height)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = build_inception_model((img_width, img_height, 3), len(os.listdir(data_dir)))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

# Example usage for predicting emotions
image_path = '/Users/eunjincho/Documents/workspaces/data_geeks/datasets/anger/anger1.jpg'
input_image = preprocess_image(image_path, img_width, img_height)
input_image = np.expand_dims(input_image, axis=0)
predicted_probs = model.predict(input_image)
predicted_emotion_index = np.argmax(predicted_probs)
emotion_classes = sorted(os.listdir(data_dir))
predicted_emotion = emotion_classes[predicted_emotion_index]
print("Predicted emotion:", predicted_emotion)


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


Epoch 1/10
9/9 [==============================] - 416s 49s/step - loss: 35.3939 - accuracy: 0.1250 - val_loss: 3.8517 - val_accuracy: 0.1000
Epoch 2/10
9/9 [==============================] - 271s 31s/step - loss: 2.9600 - accuracy: 0.1607 - val_loss: 2.0055 - val_accuracy: 0.2286
Epoch 3/10
9/9 [==============================] - 266s 30s/step - loss: 2.0782 - accuracy: 0.1643 - val_loss: 2.0066 - val_accuracy: 0.2000
Epoch 4/10
9/9 [==============================] - 271s 31s/step - loss: 1.8668 - accuracy: 0.2929 - val_loss: 1.9672 - val_accuracy: 0.1000
Epoch 5/10
9/9 [==============================] - 248s 28s/step - loss: 1.7676 - accuracy: 0.3536 - val_loss: 1.9145 - val_accuracy: 0.2143
Epoch 6/10
9/9 [==============================] - 269s 30s/step - loss: 1.6226 - accuracy: 0.4429 - val_loss: 1.9281 - val_accuracy: 0.2143
Epoch 7/10
9/9 [==============================] - 263s 30s/step - loss: 1.5173 - accuracy: 0.4714 - val_loss: 1.9120 - val_accuracy: 0.2143
Epoch 8/10
9/9 [===